In [1]:
!pip install efficientnet
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import efficientnet.tfkeras as efn  # Use tf.keras EfficientNet implementation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 831.9 kB/s eta 0:00:00a 0:00:01


2024-06-29 07:24:18.001539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 07:24:18.001685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 07:24:18.289274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
warnings.filterwarnings("ignore")
clear_output(wait=True)
print("Import Process Completed with exit status 1")

Import Process Completed with exit status 1


In [3]:
Test_Directory = "/kaggle/input/ready-leaf-disease-detection-dataset/Leaf Data/test"
Train_Directory = "/kaggle/input/ready-leaf-disease-detection-dataset/Leaf Data/train"
Validation_Directory = "/kaggle/input/ready-leaf-disease-detection-dataset/Leaf Data/validation"
Batch_Size = 20
Epochs = 18
Image_Size = (256, 256)
Number_Of_Classes = 44

In [4]:
# Use ImageDataGenerator without augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
Data_Generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)
train_Gen = Data_Generator.flow_from_directory(
    directory=Train_Directory,
    target_size=Image_Size,
    batch_size=Batch_Size,
    class_mode='categorical'
)

Validation_Gen = Data_Generator.flow_from_directory(
    directory=Validation_Directory,
    target_size=Image_Size,
    batch_size=Batch_Size,
    class_mode='categorical'
)

Test_Gen = Data_Generator.flow_from_directory(
    directory=Test_Directory,
    target_size=Image_Size,
    batch_size=Batch_Size,
    class_mode='categorical'
)

Found 38104 images belonging to 44 classes.
Found 9458 images belonging to 44 classes.
Found 5741 images belonging to 44 classes.


In [5]:
# For feature extraction (no top layer)
Base_Model = efn.EfficientNetB7(weights='imagenet', include_top=False)
Base_Model.trainable = False
Input_Layer = keras.Input(shape=(256, 256, 3))
X = Base_Model(Input_Layer, training=False)
X = layers.GlobalAveragePooling2D()(X)
Output_Layer = layers.Dense(Number_Of_Classes, activation="softmax")(X)
Model = keras.Model(Input_Layer, Output_Layer)
Model.summary()
Model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
Model.fit(train_Gen, validation_data=Validation_Gen, epochs=Epochs)

258434480/258434480 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnet-b7 (Functional)    │ (None, 8, 8, 2560)     │    64,097,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2560)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 44)             │       112,684 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,210,364 (244.94 MB)

 Trainable params: 112,684 (440.17 KB)

...

1906/1906 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.6144 - loss: 1.4604

W0000 00:00:1719646514.406513     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1906/1906 ━━━━━━━━━━━━━━━━━━━━ 713s 305ms/step - accuracy: 0.6145 - loss: 1.4601 - val_accuracy: 0.8694 - val_loss: 0.4403
Epoch 2/18


W0000 00:00:1719646617.933769     131 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1906/1906 ━━━━━━━━━━━━━━━━━━━━ 442s 232ms/step - accuracy: 0.8221 - loss: 0.5938 - val_accuracy: 0.8929 - val_loss: 0.3394
Epoch 3/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 442s 232ms/step - accuracy: 0.8452 - loss: 0.4942 - val_accuracy: 0.9135 - val_loss: 0.2792
Epoch 4/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8556 - loss: 0.4536 - val_accuracy: 0.9104 - val_loss: 0.2702
Epoch 5/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8666 - loss: 0.4160 - val_accuracy: 0.9211 - val_loss: 0.2416
Epoch 6/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8676 - loss: 0.4054 - val_accuracy: 0.9298 - val_loss: 0.2240
Epoch 7/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8748 - loss: 0.3807 - val_accuracy: 0.9266 - val_loss: 0.2193
Epoch 8/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8798 - loss: 0.3718 - val_accuracy: 0.9268 - val_loss: 0.2224
Epoch 9/18
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 441s 231ms/step - accuracy: 0.8

In [7]:
hin=Model.evaluate(Test_Gen)
print(f"Accuracy: {hin[1]}\nLoss: {hin[0]}")

288/288 ━━━━━━━━━━━━━━━━━━━━ 53s 184ms/step - accuracy: 0.9040 - loss: 0.2775
Accuracy: 0.9075074195861816
Loss: 0.2666785418987274


In [11]:
from sklearn.metrics import classification_report
print(f"Accuracy : {hin[1]*100}\nLoss : {hin[0]*100}")
Model.save("LeafDiseaseDetection.weights.h5")
Model.save_weights("LeafDiseaseDetection_.weights.h5")

Accuracy : 90.75074195861816
Loss : 26.66785418987274
